## metergroup.py

### draw_wiring_graph_2
Esta función presenta problemas con Matplotlib 3.3.0 (por defecto en NILMTK), por lo que se recomienda reemplazarla por versiones 3.1.x en caso de usarla (fuente: https://github.com/nilmtk/nilmtk/issues/868).

Se ha modificado la función original para poder graficar correctamente las etiquetas, dando paso a `draw_wiring_graph_2`

In [ ]:
    def draw_wiring_graph_2(self, show_meter_labels=True):
        graph = self.wiring_graph()
        
        try:
            # Try using graphviz layout...
            pos = nx.drawing.nx_agraph.graphviz_layout(graph, prog='dot')
            used_graphviz = True
        except:
            # ...and fallback to shell layout if graphviz is not installed or
            # doesn't work
            pos = nx.shell_layout(graph)
            used_graphviz = False
            
        meter_labels = {}
        if show_meter_labels:
            meter_labels = {meter: meter.label() for meter in graph.nodes()}
            for meter, name in meter_labels.items():
                x, y = pos[meter]

                if used_graphviz:
                    if meter.is_site_meter():
                        delta_y = 5
                    else:
                        delta_y = -5
                    
                    plt.text(x, y + delta_y, s=name, bbox=dict(facecolor='red', alpha=0.5), horizontalalignment='center')
       

### proportion_of_energy_submetered_2
Proporción de energía submedida respecto a la medición agregada.

Se ha modificado la función original para poder graficar correctamente las etiquetas, dando paso a `proportion_of_energy_submetered_2` en donde se pueden especificar por separado el medidor principal y los submedidores.

In [ ]:
    def proportion_of_energy_submetered_2(self, mains, downstream_meters,**loader_kwargs):
        """
        Returns
        -------
        float [0,1] or NaN if mains total_energy == 0
        """
        print("Running MeterGroup.proportion_of_energy_submetered...")
        #mains = self.mains() #
        #downstream_meters = self.meters_directly_downstream_of_mains() #
        proportion = 0.0
        verbose = loader_kwargs.get('verbose')
        all_nan = True
        for m in downstream_meters.meters:
            if verbose:
                print("Calculating proportion for", m)
            prop = m.proportion_of_energy(mains, **loader_kwargs)
            if not np.isnan(prop):
                proportion += prop
                all_nan = False
            if verbose:
                print("   {:.2%}".format(prop))
        
        if all_nan:
            proportion = np.NaN
        return proportion

### fraction_per_meter_2
Fracción de energía submedida respecto a la medición agregada.

Se ha modificado la función original para poder obtener la fracción de energía no submedida ('others'), dando paso a `fraction_per_meter_2`.

In [ ]:
    def fraction_per_meter_2(self, **load_kwargs):
        """Fraction of energy per meter.

        Return pd.Series.  Index is meter.instance.  
        Each value is a float in the range [0,1].
        """
        energy_per_meter = self.submeters().energy_per_meter(**load_kwargs).max()
        mains_energy = self.mains().total_energy(**load_kwargs).max()
        t_energy = energy_per_meter.sum()
        energy_per_meter=energy_per_meter.append(pd.Series(mains_energy-t_energy, index=['others']))
        
        return energy_per_meter / mains_energy